# Big Data

UC Berkeley Extension Data Analytics Boot Camp Module 16 Challenge

---
---


## Objectives

In this assignment, we were tasked with running the ETL process on the cloud, using Spark. We utilized PySpark to run statistical analysis on a database of US-based Amazon.com reviews for cameras. 


We first created our tables in the RDS database. We then extracted the data from the S3 bucket and loaded it into a dataframe. Once the data was in a DataFrame, we transformed it to fit the desired schema (from schema.sql). To complete the ETL process, we loaded the DataFrames that correspond to tables into an RDS instance.


Once the ETL process was completed, we performed statistical analyses in PySpark to determine if the Vine reviews were unbiased.

---
---

## Setup

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.'
os.environ['SPARK_VERSION']=spark_version

# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [ ]:
# set up postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-10-15 21:26:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.02MB/s    in 0.9s    

2020-10-15 21:26:43 (1.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# pyspark setup
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ChallengeETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

---
---

## ETL Process

---

### Extract

In [1]:
# link to camera reviews: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz

# structure:
# marketplace       - 2 letter country code of the marketplace where the review was written.
# customer_id       - Random identifier that can be used to aggregate reviews written by a single author.
# review_id         - The unique ID of the review.
# product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
#                     for the same product in different countries can be grouped by the same product_id.
# product_parent    - Random identifier that can be used to aggregate reviews for the same product.
# product_title     - Title of the product.
# product_category  - Broad product category that can be used to group reviews 
#                     (also used to group the dataset into coherent parts).
# star_rating       - The 1-5 star rating of the review.
# helpful_votes     - Number of helpful votes.
# total_votes       - Number of total votes the review received.
# vine              - Review was written as part of the Vine program.
# verified_purchase - The review is on a verified purchase.
# review_headline   - The title of the review.
# review_body       - The review text.
# review_date       - The date the review was written.

url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
raw_camera_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)


NameError: ignored

---

### Transform

In [2]:
# Inspect

raw_camera_df.show(5)

print(f"There are ", raw_camera_df.count(), " rows in this dataset \n")

raw_camera_df.printSchema()

NameError: ignored

The schema of the data does not our desired schema. However, the data types match. Next, we need to check for null values within the reviews. Once the null values are removed, we can work on matching the schemas.

In [ ]:
# Drop nulls

camera_df = raw_camera_df.dropna()

print(f"There are ", camera_df.count()," values once the nulls are dropped.\n")

camera_df.show(5)

We found that there were 125 null values. The camera_df DataFrame houses the cleaned, non-null data that we will further transform.

Desired Schema:

--- 

review_id_table:

|Column|Data Type|
|-----|-------|
|review_id|text|
|customer_id|integer|
|product_id|text|
|product_parent|integer|
|review_date|date (yyyy-mm-dd)|

---

products:

|Column|Data Type|
|------|--------|
|product_id (unique)|text|
|product_text|text|

---

customers:

|Column|Data Type|
|-----|-------|
|customer_id (unique)|integer|
|customer_count|integer|

---

vine_table: 

|Column|Data Type|
|-----|-------|
|review_id|text|
|star_rating|integer|
|helpful_votes|integer|
|total_votes|integer|
|vine|text|

---





In [ ]:
# create review_id_table DF:

review_id_table_df = camera_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_table_df.show(5)

In [ ]:
# create products DF:

products_df = camera_df.select(['product_id', 'product_title']).distinct()
products_df.show(5)

In [ ]:
# create customers DF:

# First we must create the customer_count column (as integers)

# Load in a sql function to use column functions
from pyspark.sql.functions import col

camera_df = camera_df.withColumn('customer_count', sum((camera_df['customer_id']>0).cast('integer') for col in camera_df.groupby().count())) 


customers_df = camera_df.select(['customer_id','customer_count'])
customers_df.show()

In [ ]:
# create vine_table DF:

vine_table_df = camera_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes','vine'])
vine_table_df.show(5)

Now, all our tables match the schema we want. We are done with the transformation process, and are ready to move on to the loading process.

---
---

### Load:



In [ ]:
# Setup RDS
mode = "append"
jdbc_url="jdbc:postgresql://AWS_Challenge16.c4brwagnxkub.us-east-2.rds.amazonaws.com:5432/challenge16"
config = {"user":"postgres", 
          "password": "Module16", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Load the DataFrames as tables in RDS

# review_id_table:
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

# products:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

# customers:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

# vine_table:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

We have sucessfully completed the loading process. Now, we can move on to analysis.

---
---

## Analysis

The goal of our analysis is to determine whether Vine reviews are biased or not for the cameras listed on Amazon. We will do this via various statistical methods.

First, we need to separate the Vine reviews from the non-Vine reviews.



In [ ]:
# create DataFrames for Vine and Non-Vine reviews


vine_only_df = vine_table_df.filter(vine_table_df["vine"] == "N")
non_vine_df = vine_table_df.filter(vine_table_df["vine"] == "Y")

vine_only_df.show(5)
non_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2H4QESQ10P8EN|          5|            4|          5|   Y|
|  RY5OEUWERD2Z|          4|            5|          5|   Y|
|R3GNSMDV7WZGKP|          4|            1|          1|   Y|
|R1RI2CNUVSAOU6|          4|            0|          0|   Y|
|R3UX0VTCK4SUJX

Now that we have separated the data into Vine and Non-Vine reviews, we can perform statistical analysis on both DataFrames to determine whether there is bias in the Vine reviews.

In [ ]:
# calculate the number of reviews in each set

print(f"There are ",vine_only_df.count()," paid reviews, compared to ", non_vine_df.count()," non-paid reviews.")

There are  1793966  paid reviews, compared to  7883  non-paid reviews.


We can see that a majority of the reviews are from the Vine cohort by a very large margin. While this does not imply any biases, it is worth looking into in more detail.

In [ ]:
# calculate the average rating

from pyspark.sql.functions import avg

print("In the Non-Vine data set, the average rating is:")
non_vine_df.select(avg("star_rating")).show()

print("In the Vine data set, the average rating is:")
vine_only_df.select(avg("star_rating")).show()

In the Non-Vine data set, the average rating is:
+-----------------+
| avg(star_rating)|
+-----------------+
|4.128250665990105|
+-----------------+

In the Vine data set, the average rating is:
+-----------------+
| avg(star_rating)|
+-----------------+
|4.127021916803328|
+-----------------+



We see that the average review score for both Vine and Non-Vine data sets are both around 4.13 out of 5. Based on this alone, we cannot claim that the Vine data set shows bias. 

Now we can compare each set's share of 1 to 5 star ratings.


In [ ]:
# See a breakdown of ratings for each set.

print(f"", "%.1f"% (vine_only_df.filter("star_rating=1").count()/vine_only_df.count()*100),"% of the Vine reviews were rated 1 star, compared to ", "%.1f"% (non_vine_df.filter("star_rating=1").count()/non_vine_df.count()*100),"% of Non-Vine 1 star ratings.")
print(f"", "%.1f"% (vine_only_df.filter("star_rating=2").count()/vine_only_df.count()*100),"% of the Vine reviews were rated 2 stars, compared to ", "%.1f"% (non_vine_df.filter("star_rating=2").count()/non_vine_df.count()*100),"% of Non-Vine 2 star ratings.")
print(f"", "%.1f"% (vine_only_df.filter("star_rating=3").count()/vine_only_df.count()*100),"% of the Vine reviews were rated 3 stars, compared to ", "%.1f"% (non_vine_df.filter("star_rating=3").count()/non_vine_df.count()*100),"% of Non-Vine 3 star ratings.")
print(f"", "%.1f"% (vine_only_df.filter("star_rating=4").count()/vine_only_df.count()*100),"% of the Vine reviews were rated 4 stars, compared to ", "%.1f"% (non_vine_df.filter("star_rating=4").count()/non_vine_df.count()*100),"% of Non-Vine 4 star ratings.")
print(f"", "%.1f"% (vine_only_df.filter("star_rating=5").count()/vine_only_df.count()*100),"% of the Vine reviews were rated 5 stars, compared to ", "%.1f"% (non_vine_df.filter("star_rating=5").count()/non_vine_df.count()*100),"% of Non-Vine 5 star ratings.")


 9.5 % of the Vine reviews were rated 1 star, compared to  1.8 % of Non-Vine 1 star ratings.
 5.0 % of the Vine reviews were rated 2 stars, compared to  4.5 % of Non-Vine 2 star ratings.
 7.8 % of the Vine reviews were rated 3 stars, compared to  14.4 % of Non-Vine 3 star ratings.
 18.6 % of the Vine reviews were rated 4 stars, compared to  37.4 % of Non-Vine 4 star ratings.
 59.1 % of the Vine reviews were rated 5 stars, compared to  41.8 % of Non-Vine 5 star ratings.


We see that there are more 5 star reviews for the Vine reviews than for the Non-Vine reviews. However, there are also more 1 star reviews as well. 

#### Conclusion:

Given the much larger number of responses in the Vine set than the non-paid Non-Vine set, we would expect to see the average ratings for the Vine set to be much higher than the Non-Vine set. However, this does not appear to be the case. We can conclude that the reviews for the Camera dataset do not show any significant bias towards the paid Vine reviews, and they are trustworthy.